<a href="https://colab.research.google.com/github/chiranjeet14/ML_Journey/blob/master/Hackerearth-Predict-the-genetic-disorders/1_genetic_testing_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import io
import gc
import time
from datetime import date

# settings
import warnings
warnings.filterwarnings("ignore")
gc.enable()

In [ ]:
!pip3 install pandas-profiling==2.8.0 > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gDrivePath = '/content/drive/MyDrive/Datasets/Hackerearth_genetic_testing/dataset/'

In [ ]:
df_train = pd.read_csv(gDrivePath+'train.csv')
df_test = pd.read_csv(gDrivePath+'test.csv')

In [ ]:
df_train.shape

(22083, 45)

In [ ]:
df_test.shape

(9465, 43)

In [ ]:
df_train.sample(3)

,Patient Id,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Patient First Name,Family Name,Father's name,Mother's age,Father's age,Institute Name,Location of Institute,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Test 1,Test 2,Test 3,Test 4,Test 5,Parental consent,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Place of birth,Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
16833,PID0x7669,11.0,Yes,Yes,No,Yes,4.842162,Lindsey,Wilson,Galo,23.0,38.0,Arbour Hospital,"59 TOWNSEND ST\nROXBURY, MA 02119\n(42.3185628...",Alive,Normal (30-60),Normal,0.0,0.0,0.0,1.0,0.0,Yes,High,Male,Not available,Not applicable,Institute,Yes,Yes,Yes,Yes,No,Yes,4.0,Multiple,8.105053,abnormal,0.0,0.0,0.0,1.0,0.0,Mitochondrial genetic inheritance disorders,Mitochondrial myopathy
8332,PID0xcf0,NaN,Yes,Yes,Yes,Yes,4.634455,Roger,Marcus,Jahquel,NaN,NaN,NaN,"125 PARKER HILL AV\nJAMAICA PLAIN, MA 02120\n(...",Deceased,Normal (30-60),Tachycardia,0.0,NaN,0.0,1.0,NaN,Yes,Low,Female,NaN,None,Institute,NaN,No,Not applicable,NaN,Yes,Yes,4.0,Multiple,10.363715,inconclusive,1.0,1.0,1.0,NaN,0.0,Single-gene inheritance diseases,Cystic fibrosis
7937,PID0x1a54,8.0,No,Yes,Yes,No,5.158115,Mildred,NaN,Khayri,41.0,NaN,Not applicable,-,Alive,Normal (30-60),Normal,0.0,0.0,0.0,1.0,0.0,Yes,Low,Female,No,Not applicable,Home,Yes,Yes,Yes,Not applicable,No,Yes,2.0,Singular,10.561408,slightly abnormal,1.0,1.0,0.0,NaN,1.0,NaN,Leigh syndrome


## Preprocessing

### Drop unnecessary columns

In [ ]:
drop_cols = ["Patient Id", "Patient First Name", "Family Name", "Father's name", "Institute Name", "Location of Institute", "Parental consent", "Place of birth", "Test 1", "Test 2", "Test 3", "Test 4", "Test 5"]

df_train.drop(drop_cols, axis=1, inplace=True, errors='ignore')
df_test.drop(drop_cols, axis=1, inplace=True, errors='ignore')

In [ ]:
# drop rows from train data where target values are missing
df_train.dropna(how='any', subset=["Genetic Disorder", "Disorder Subclass"], inplace=True)

In [ ]:
df_train.sample(3)

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
20639,14.0,Yes,Yes,Yes,No,4.626668,47.0,30.0,Deceased,Normal (30-60),Normal,Low,Male,No record,Yes,Yes,Yes,-,Not applicable,Yes,Yes,4.0,Singular,8.521529,normal,1.0,0.0,0.0,1.0,1.0,Single-gene inheritance diseases,Cystic fibrosis
17827,5.0,No,Yes,NaN,Yes,4.814127,NaN,NaN,Deceased,Normal (30-60),NaN,NaN,Ambiguous,Yes,Yes,No,Yes,Not applicable,-,Yes,No,NaN,Singular,6.539849,abnormal,NaN,0.0,NaN,1.0,1.0,Single-gene inheritance diseases,Cystic fibrosis
16192,NaN,Yes,Yes,No,Yes,4.640285,28.0,53.0,Alive,NaN,Normal,High,Male,No,Not applicable,Yes,No,No,Not applicable,Yes,Yes,NaN,Multiple,8.885602,NaN,1.0,0.0,1.0,0.0,1.0,Multifactorial genetic inheritance disorders,Diabetes


In [ ]:
# from pandas_profiling import ProfileReport
# report = ProfileReport(df_train, title="Pandas Profiling Report", explorative=True)
# report.to_file("genetic_testing.html")

In [ ]:
# report

In [ ]:
def get_categorical_features(dataFrame):
    categorical_feats = [
        f for f in dataFrame.columns if dataFrame[f].dtype == 'object'
    ]
    return categorical_feats

def get_non_categorical_features(dataFrame, categorical_feats):
    non_categorical_features  = [
        f for f in dataFrame.columns if f not in categorical_feats
    ]
    return non_categorical_features

def remove_element_from_list(list_of_elements, element):
    if element in list_of_elements: list_of_elements.remove(element)
    return list_of_elements

In [ ]:
target_feature = ['Genetic Disorder', 'Disorder Subclass']
categorical_features = get_categorical_features(df_train)
categorical_features = remove_element_from_list(categorical_features, target_feature)

non_categorical_features = get_non_categorical_features(df_train, categorical_features)
non_categorical_features = remove_element_from_list(non_categorical_features, target_feature)

In [ ]:
# categorical_features

In [ ]:
# non_categorical_features

In [ ]:
print("TOTAL COLUMNS : ", len(df_train.columns))
print("CATEGORICAL COLUMNS : ", len(categorical_features))
print("NON-CATEGORICAL COLUMNS : ", len(non_categorical_features))

TOTAL COLUMNS :  32
CATEGORICAL COLUMNS :  21
NON-CATEGORICAL COLUMNS :  11


In [ ]:
def convert_to_categorical(dataFrame, field):
    dataFrame[field] = dataFrame[field].astype('category')
    return dataFrame

def convert_to_numeric(dataFrame, field):
    dataFrame[field] = dataFrame[field].apply(pd.to_numeric, errors='coerce')
    return dataFrame

### Keeping aside target columns

In [ ]:
target_feature = ['Genetic Disorder', 'Disorder Subclass']

df_target = pd.DataFrame(df_train, columns = target_feature)
df_train.drop(target_feature, axis=1, inplace=True, errors='ignore')

### Merge train and test for preprocessing

In [ ]:
df_merged = pd.concat([df_test.assign(source=0), df_train.assign(source=1)])
# df_test = 0
# df_train = 1

In [ ]:
df_merged.sample(2)

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,source
5455,0.0,Yes,No,Yes,No,5.128057,45.0,20.0,Alive,NaN,NaN,Low,Female,-99,Not applicable,No,No,Not applicable,-99,Yes,No,3.0,-99,-99.000000,slightly abnormal,1.0,0.0,1.0,1.0,1.0,0
15981,13.0,Yes,No,NaN,No,4.894106,22.0,NaN,Deceased,Normal (30-60),Tachycardia,Low,Ambiguous,Yes,Yes,No,NaN,-,-,No,Yes,1.0,Singular,8.617152,NaN,0.0,1.0,1.0,NaN,0.0,1


### Encoding categorical features to numeric while preserving nan values

In [ ]:
categorical_features = get_categorical_features(df_merged)
non_categorical_features = get_non_categorical_features(df_merged, categorical_features)

In [ ]:
categorical_features

["Genes in mother's side",
 'Inherited from father',
 'Maternal gene',
 'Paternal gene',
 'Status',
 'Respiratory Rate (breaths/min)',
 'Heart Rate (rates/min',
 'Follow-up',
 'Gender',
 'Birth asphyxia',
 'Autopsy shows birth defect (if applicable)',
 'Folic acid details (peri-conceptional)',
 'H/O serious maternal illness',
 'H/O radiation exposure (x-ray)',
 'H/O substance abuse',
 'Assisted conception IVF/ART',
 'History of anomalies in previous pregnancies',
 'Birth defects',
 'Blood test result']

In [ ]:
non_categorical_features

['Patient Age',
 'Blood cell count (mcL)',
 "Mother's age",
 "Father's age",
 'No. of previous abortion',
 'White Blood cell count (thousand per microliter)',
 'Symptom 1',
 'Symptom 2',
 'Symptom 3',
 'Symptom 4',
 'Symptom 5',
 'source']

In [ ]:
# ref: https://krrai77.medium.com/using-fancyimpute-in-python-eadcffece782

from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()

#list of categorical variables
cat_cols = categorical_features

#This function will encode non-null data and replace it in the original data
def ordinalencode(dataframe):
  nonulls = np.array(dataframe.dropna())
  impute_reshape = nonulls.reshape(-1,1)
  impute_ordinal = encoder.fit_transform(impute_reshape)
  dataframe.loc[dataframe.notnull()] = np.squeeze(impute_ordinal)
  return dataframe

#encoding all the categorical data in the data set through looping
for columns in cat_cols:
  ordinalencode(df_merged[columns])

In [ ]:
df_merged.sample(5)

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,source
6750,3.0,0.0,0,1,0.0,4.959893,23.0,25.0,1.0,2,2,2,3,3,2,1,2,3,4,1,1,4.0,2,5.554211,4,1.0,0.0,0.0,1.0,1.0,0
104,8.0,0.0,NaN,1,1.0,4.746169,34.0,52.0,1.0,NaN,NaN,1,2,4,0,2,2,4,4,1,1,-99.0,0,10.836248,0,1.0,1.0,0.0,0.0,1.0,0
9049,1.0,1.0,0,NaN,1.0,4.809788,19.0,36.0,1.0,0,2,2,3,0,1,2,0,3,3,2,1,1.0,1,3.000000,4,1.0,1.0,0.0,1.0,1.0,0
16765,0.0,1.0,0,0,0.0,5.176665,24.0,59.0,0.0,2,1,1,1,2,3,1,1,3,0,2,1,2.0,2,4.108044,4,0.0,0.0,0.0,1.0,0.0,1
8468,6.0,0.0,1,0,0.0,4.603311,51.0,57.0,0.0,2,1,2,2,1,3,1,1,0,2,2,1,1.0,2,12.000000,3,0.0,1.0,0.0,NaN,1.0,1


### Replacing NaN values - MICE imputation

In [ ]:
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer as MICE

# Create a copy of my original dataset
dataset_impute = df_merged.copy()

# Apply MICE
dataset_impute_complete = MICE(max_iter=150, verbose=1, initial_strategy='most_frequent',random_state=10).fit_transform(dataset_impute.values)

# Turning into df again
df_merged = pd.DataFrame(data=dataset_impute_complete, columns=dataset_impute.columns, index=dataset_impute.index)

[IterativeImputer] Completing matrix with shape (27512, 31)
[IterativeImputer] Change: 148.6737680075267, scaled tolerance: 0.099 
[IterativeImputer] Change: 1.5144384958684534, scaled tolerance: 0.099 
[IterativeImputer] Change: 0.03916912213077839, scaled tolerance: 0.099 
[IterativeImputer] Early stopping criterion reached.


In [ ]:
df_merged.sample(5)

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,source
2774,8.0,1.0,1.000000,1.000000,0.0,4.980267,46.0,31.0,0.0,1.004949,1.095206,1.0,0.0,0.0,3.0,2.0,1.0,0.0,0.0,1.0,1.0,1.0,2.0,-99.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4925,3.0,1.0,0.000000,1.000000,0.0,4.926631,50.0,29.0,0.0,2.000000,2.000000,1.0,3.0,1.0,3.0,2.0,2.0,3.0,0.0,2.0,1.0,3.0,2.0,4.964437,2.0,1.0,1.0,0.0,0.0,0.0,1.0
5484,1.0,0.0,0.430068,0.000000,1.0,5.058698,20.0,55.0,1.0,0.000000,0.000000,2.0,2.0,2.0,0.0,2.0,2.0,3.0,2.0,0.0,0.0,4.0,2.0,7.764864,1.0,1.0,1.0,1.0,0.0,0.0,0.0
7027,3.0,1.0,1.000000,1.000000,0.0,4.846913,37.0,38.0,0.0,1.000856,1.000000,2.0,2.0,1.0,3.0,2.0,2.0,2.0,3.0,1.0,1.0,0.0,2.0,6.767499,4.0,1.0,0.0,0.0,0.0,1.0,0.0
5736,4.0,0.0,0.000000,0.522574,0.0,4.919550,49.0,49.0,0.0,2.000000,1.055132,2.0,2.0,3.0,3.0,2.0,2.0,1.0,1.0,2.0,1.0,0.0,0.0,3.000000,3.0,0.0,0.0,1.0,1.0,1.0,0.0


In [ ]:
# split again into test and train
# df_test, df_train = df_merged[df_merged["source"].eq("df_test")], df_merged[df_merged["source"].eq("df_train")]
df_test, df_train = df_merged[df_merged["source"].eq(0.0)], df_merged[df_merged["source"].eq(1.0)]
df_train.drop(["source"], axis=1, inplace=True, errors='ignore')
df_test.drop(["source"], axis=1, inplace=True, errors='ignore')

In [ ]:
df_train = pd.concat([df_train, df_target], axis=1)
df_train.sample(5)

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
17426,3.0,1.0,0.0,1.0,0.0,4.853941,27.0,27.0,0.0,2.0,2.0,2.0,1.0,3.0,3.0,2.0,1.0,0.0,2.0,1.0,2.0,1.0,2.0,4.248258,1.0,1.0,1.0,1.0,0.0,1.0,Single-gene inheritance diseases,Cystic fibrosis
20467,6.0,0.0,1.0,1.0,1.0,4.554800,29.0,31.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,0.0,0.0,1.0,2.0,2.0,1.0,5.582029,3.0,1.0,1.0,1.0,0.0,1.0,Single-gene inheritance diseases,Cystic fibrosis
19020,7.0,1.0,0.0,0.0,0.0,4.783431,43.0,50.0,0.0,1.0,2.0,2.0,1.0,2.0,3.0,2.0,1.0,4.0,0.0,1.0,1.0,2.0,1.0,8.015490,2.0,1.0,1.0,1.0,1.0,1.0,Multifactorial genetic inheritance disorders,Diabetes
18206,9.0,1.0,1.0,1.0,1.0,4.913101,31.0,37.0,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1.0,1.0,4.0,2.0,1.0,1.0,2.0,1.0,4.598497,4.0,1.0,0.0,1.0,0.0,1.0,Multifactorial genetic inheritance disorders,Diabetes
4506,10.0,0.0,1.0,1.0,1.0,4.660750,31.0,26.0,0.0,1.0,1.0,1.0,3.0,1.0,3.0,1.0,2.0,2.0,0.0,2.0,1.0,4.0,1.0,11.061508,1.0,1.0,1.0,0.0,1.0,1.0,Multifactorial genetic inheritance disorders,Alzheimer's


In [ ]:
print("NaN Values:",df_train.isna().any().sum())

NaN Values: 0


In [ ]:
print("NaN Values:",df_test.isna().any().sum())

NaN Values: 0


In [ ]:
# shape of data
print("Train shape: ",df_train.shape)
print("Test shape: ",df_test.shape)

Train shape:  (18047, 32)
Test shape:  (9465, 30)


In [ ]:
# save preprocessed files
df_train.to_csv(gDrivePath+'train_preprocessed.csv', index=False)
df_test.to_csv(gDrivePath+'test_preprocessed.csv', index=False)